# Load Bridges2 Modules and Set Environment Variables

In [ ]:
%set_env SHELL=/bin/bash

In [ ]:
import sys
sys.path.append("/usr/share/lmod/lmod/init")
sys.path.append("/jet/home/wnw36/.conda/envs/icomse-dft/lib/python3.1/site-packages/")
from env_modules_python import module
module('load', 'intelmpi')
module('load', 'QuantumEspresso')

In [ ]:
import os
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun -np 6 pw.x -in PREFIX.pwi > PREFIX.pwo"
os.environ['OMP_NUM_THREADS'] = "2"

# SCF Calculation for Metallic Surfaces

In [ ]:
from ase.visualize import view
from ase.build import fcc111

atoms = fcc111('Al', size=(2,2,4), vacuum=7.5) # Vacuum added on both sides! 15 Ang. Vacuum total

view(atoms, viewer="x3d")

In [ ]:
from ase.calculators.espresso import Espresso

pseudopotentials = {
    "Al":"Al.pbe-n-kjpaw_psl.1.0.0.UPF",
    "H":"H_ONCV_PBE-1.0.oncvpsp.upf"
}

input_data = {
    'system': {
        'ecutwfc': 20,
        'ecutrho': 160,
        'occupations': 'smearing',
        'smearing':'mp',
        'degauss':0.02
    },
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos'
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta':0.2,
        'mixing_mode': 'local-TF'
    }
} 


calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(2,2,1),
    koffset=(1, 1, 0))

atoms.calc = calc


In [ ]:
energy_slab = atoms.get_potential_energy()
print(f"E = {energy_slab:.3f}")

# Adding Adsorbates to Surfaces

In [ ]:
from ase.build import add_adsorbate, molecule
add_adsorbate(atoms, molecule("H2"), 2.75, 'ontop')
view(atoms, viewer="x3d")

In ASE, Atoms when "add_adsorbate" and "surface" are used get assigned tags. Each layer of the slab is numbered 1, 2, 3, etc. while adsorbates are numbered 0.

In [ ]:
print(atoms.get_tags())

In [ ]:
idx = [atom.index for atom in atoms if atom.tag == 0]
atoms[idx]

# Slab + Adsorbate SCF

In [ ]:
energy_system = atoms.get_potential_energy()
print(f"E = {energy_system:.3f}")

# Adsorbate SCF

In [ ]:
idx_Al = [atom.index for atom in atoms if atom.tag != 0]
del atoms[idx_Al]
energy_probe = atoms.get_potential_energy()

# Binding Energy Calculation

In [ ]:
print(f"Binding Energy = {energy_system - energy_slab - energy_probe:.3f} eV" )